# 兩層Convolution network分類


In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 讀取資料
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# x is the placeholder of image
# y_ is the placeholder of label
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

W1012 19:51:38.402569  2704 deprecation.py:323] From <ipython-input-2-42cb7771f94f>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1012 19:51:38.405562  2704 deprecation.py:323] From A:\Anaconda3\envs\tensorflowenv\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1012 19:51:38.407556  2704 deprecation.py:323] From A:\Anaconda3\envs\tensorflowenv\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for up

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1012 19:51:38.944085  2704 deprecation.py:323] From A:\Anaconda3\envs\tensorflowenv\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1012 19:51:38.977965  2704 deprecation.py:323] From A:\Anaconda3\envs\tensorflowenv\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1012 19:51:39.126027  2704 deprecation.py:323] From A:\Anaconda3\envs\tensorflowenv\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be rem

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


因為要用Convolution，所以要將784維的x轉換成28\*28維的圖片格式

In [3]:
# 將單張圖片從784-D轉換成28*28-D
x_image = tf.reshape(x, [-1, 28, 28, 1]) # -1表示第一維的大小由x自動決定

## 第一層Convolution

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape= shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                             strides=[1,2,2,1],
                             padding='SAME')

`weight_variable()`傳回一個指定形狀的變數，並且自動以截斷常態分佈初始化  
`bias_variable()`傳回一個指定形狀的變數，設定初始值為0.1  
這兩個函數可以用來建立<strong>kernel</strong>和<strong>bias</strong>  

In [5]:
# conv layer 1
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
# 1st conv
h_conv1 = tf.nn.relu(
              conv2d(x_image, W_conv1) + b_conv1 # 對圖片和W1 做Convolution
          )
# 1st pooling
h_pool1 = max_pool_2x2(h_conv1)

`h_conv1`是在做真正的Convolution計算，並用**Relu**作為Active function  
`h_pool1`是在做池化動作  
通常一個Convolution層都會包含：Convolution、Active function、pooling，但有的時候也會去掉最後的池化動作

## 第二層Convolution

In [6]:
# conv layer 2
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
# 2nd conv
h_conv2 = tf.nn.relu(
              conv2d(h_conv1 , W_conv2) + b_conv2 # 對第一層出來的做Convolution
          )
# 2nd conv
h_pool2 = max_pool_2x2(h_conv2)

## 全連接層

In [7]:
# full-connect layer 1: 1024 output
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
# 1st full connect
h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*64]) # make the pooling layer into flat
h_fc1 = tf.nn.relu(
            tf.matmul(h_pool2_flat, W_fc1) + b_fc1
        )
# Dropout
# keep_prob is a placeholder, training:"0.5", testing:"1"
# 新版本會移除keep_prob，所以用rate代替，rate = 1-keep_prob
rate = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W1012 19:51:40.285192  2704 deprecation.py:506] From <ipython-input-7-fca84b960c24>:14: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## 定義CrossEntropy和Train_step

In [9]:
# 直接用tf.nn.softmax_cross_entropy_with_logits而非Softmax再計算的方法
cross_entropy = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(
                        labels=y_, logits=y_conv
                    )
                )
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

W1012 19:51:40.342307  2704 deprecation.py:323] From <ipython-input-9-66052f162593>:4: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
correct_prediction = tf.equal(
                        tf.argmax(y_conv,1), tf.argmax(y_,1)
                     )
accuracy = tf.reduce_mean(
               tf.cast(correct_prediction, tf.float32)
           )

## 建立Session

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
for i in range(2000):
    batch = mnist.train.next_batch(50)
    # 每100步報告一次準確率
    if i %100 == 0:
        train_accuray = accuracy.eval(feed_dict={
                            x:batch[0], y_:batch[1], keep_prob: 1.0
                        })
        print("step %d:training accuracy %g" % (i,train_accuray))
    train_step.run(feed_dict={
        x:batch[0], y_:batch[1], keep_prob: 0.5
    })

InvalidArgumentError: Incompatible shapes: [200] vs. [50]
	 [[node Equal (defined at <ipython-input-10-a95e52f5f6be>:2) ]]

Original stack trace for 'Equal':
  File "A:\Anaconda3\envs\tensorflowenv\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "A:\Anaconda3\envs\tensorflowenv\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "A:\Anaconda3\envs\tensorflowenv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "A:\Anaconda3\envs\tensorflowenv\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "A:\Anaconda3\envs\tensorflowenv\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "A:\Anaconda3\envs\tensorflowenv\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "A:\Anaconda3\envs\tensorflowenv\lib\asyncio\base_events.py", line 438, in r